# 01. 나이브 베이즈 (Naive Bayes): 확률로 찍는 분류기
### 과목: NLP Text Classification
---

## 1. "스팸일까 아닐까?" 확률로 맞혀보자!
나이브 베이즈는 **조건부 확률(Conditional Probability)**을 이용해 정답을 추측하는 아주 빠르고 강력한 알고리즘입니다.
텍스트 분류(스팸 메일 필터링, 뉴스 카테고리 분류 등)에서 오랫동안 사랑받아왔습니다.

### 🔍 핵심 아이디어: 베이즈 정리 (Bayes' Theorem)
> "B라는 증거가 나왔을 때, A일 확률은?"

$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$

- 예시: 메일에 **"무료"**라는 단어가 들어있을 때(B), 이 메일이 **스팸(A)**일 확률은?
    - $P(\text{스팸}|\text{무료}) = \frac{P(\text{무료}|\text{스팸}) \times P(\text{스팸})}{P(\text{무료})}$

## 2. 왜 "나이브(Naive, 순진한)"라고 부를까?
이 알고리즘은 **"모든 단어(특징)가 서로 독립이다"**라고 아주 순진하게 가정하기 때문입니다.
- 실제로는 '홍 길동' 처럼 단어끼리 연관이 있지만, 나이브 베이즈는 '홍'과 '길동'이 전혀 상관없이 따로따로 등장했다고 칩니다.
- **장점**: 계산이 엄청나게 단순해지고 빨라집니다. 그런데도 성능은 꽤 좋습니다!

## 3. 대표적인 3총사
1. **Gaussian NB**: 키, 몸무게 같은 연속적인 숫자 데이터일 때 (정규분포 가정)
2. **Multinomial NB**: **텍스트 데이터(단어 빈도수)** 처럼 횟수가 중요할 때 ★(오늘 배울 것)
3. **Bernoulli NB**: 있다/없다(0 또는 1) 같은 이진 데이터일 때


---
## 4. 실습 1: 아주 간단한 감성 분석 (긍정 vs 부정)
데이터가 아주 작을 때 어떻게 동작하는지 봅시다.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import pandas as pd

# 1. 초미니 데이터셋
texts = [
    "I love this movie",      # 긍정
    "This film was amazing",  # 긍정
    "I hate this movie",      # 부정
    "This movie was terrible" # 부정 (오타 수정됨: moive -> movie)
]
labels = ["pos", "pos", "neg", "neg"]

# 2. BoW (Bag of Words) 만들기
# 텍스트를 숫자(단어 등장 횟수)로 바꿔야 컴퓨터가 이해합니다.
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(texts)

# 눈으로 확인해봅시다.
print("--- 단어 사전 ---")
print(vectorizer.get_feature_names_out())
print("\n--- 문서를 숫자로 바꾼 결과 (행:문서, 열:단어) ---")
print(X_train.toarray())

## 5. 모델 학습
**MultinomialNB**는 "이 단어가 나타났을 때 긍정일 확률이 높니, 부정일 확률이 높니?"를 학습합니다.

In [ ]:
# 모델 생성 및 학습
clf = MultinomialNB()
clf.fit(X_train, labels)
print("학습 완료!")

In [ ]:
# 6. 예측해보기
test_text = ["I love this movie"] # 훈련 데이터에 있던 문장이지만 테스트용으로 써봄

# 주의: 테스트 데이터는 반드시 학습 데이터(vectorizer)를 기준으로 변환해야 합니다.
# fit_transform()이 아니라 transform()을 씁니다.
X_test = vectorizer.transform(test_text)

pred = clf.predict(X_test)
print(f"문장: '{test_text[0]}'")
print(f"예측 결과: {pred[0]} (정답!)")

In [ ]:
# 확률도 확인해볼까요?
proba = clf.predict_proba(X_test)
classes = clf.classes_

print(f"클래스 순서: {classes}")
print(f"확률 분포: {proba}")
# love 같은 긍정 단어가 있어서 pos 확률이 더 높게 나왔을 겁니다.

---
## 7. 심화: 내부 들여다보기 (Log Probability)
나이브 베이즈는 컴퓨터 계산 문제(Underflow)를 막기 위해 확률에 **Log(로그)**를 씌워서 계산합니다.
- 확률은 0~1 사이 값이라서 계속 곱하면 0.00000...으로 사라져 버리기 때문입니다.
- 로그를 씌우면 곱하기가 더하기로 바뀌어 계산이 편해집니다.

In [ ]:
# 1. 클래스별 사전 확률 (로그 값)
# 데이터셋에 긍정 2개, 부정 2개였으므로 확률은 50:50 (0.5)
# log(0.5) ≈ -0.693
print(f"클래스 사전 확률 (Log): {clf.class_log_prior_}")

# 2. 단어별 조건부 확률 (이 단어가 긍정/부정에서 나올 확률)
feature_log_prob = clf.feature_log_prob_

# 보기 좋게 표로 만듦
df_prob = pd.DataFrame(
    feature_log_prob,
    columns=vectorizer.get_feature_names_out(),
    index=clf.classes_
)
print("\n--- 단어별 Log 확률 (값이 클수록, 즉 0에 가까울수록 중요한 단어) ---")
display(df_prob)

---
## 8. 실전: 뉴스그룹 분류 (20 Newsgroups)
조금 더 복잡한 데이터로 실험해봅시다. 주제가 다른 수천 개의 뉴스를 자동으로 분류합니다.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# 1. 데이터 로드 (시간이 조금 걸릴 수 있습니다)
print("데이터 다운로드 중...")
newsgroup = fetch_20newsgroups(subset='all')
print(f"총 문서 개수: {len(newsgroup.data)}")

# 2. TF-IDF 변환
# CountVectorizer보다 좀 더 똑똑한 방식 (흔한 단어 패널티 줌)
vectorizer = TfidfVectorizer(stop_words='english') # 영어 불용어(the, a 등) 제거
X = vectorizer.fit_transform(newsgroup.data)
y = newsgroup.target

print(f"데이터 크기 (문서, 단어): {X.shape}")

# 3. 훈련/테스트 셋 분리 (8:2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("데이터 분리 완료!")

In [ ]:
# 4. 모델 학습
model = MultinomialNB()
model.fit(X_train, y_train)
print("뉴스그룹 분류기 학습 완료!")

In [ ]:
# 5. 성능 평가
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"정확도: {acc:.4f}")
print("\n--- 상세 리포트 ---")
print(classification_report(y_test, y_pred, target_names=newsgroup.target_names))

# 나이브 베이즈는 딥러닝보다 훨씬 빠르면서도 꽤 준수한 성능(약 88%)을 보여줍니다.